In [1]:
from optimization.ils import ILS_VRP
from optimization.ils_sa import ILS_SA_VRP
from optimization.ils_ts import ILS_TS_VRP
from optimization.ils_sa_ts import ILS_SA_TS_VRP
from optimization.mga import MGA_VRP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix#,tourid,idhotel

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## Outlier Sensitivity

In [5]:
hotel,tur,timematrix = generate_data(n=87,random_state=40)
dwaktu,dtarif,drating = 1,0,0

In [6]:
id_outlier = [39, 58, 62, 64, 65]

In [7]:
min_days = 1
max_days = 13

In [9]:
outlier_sensitivity_analysis = {1 : {},
                                2 : {},
                                3 : {},
                                4 : {},
                                5 : {},
                                6 : {},
                                7 : {},
                                8 : {},
                                9 : {},
                                10 : {},
                                11 : {},
                                12 : {},
                               13 : {}}

for n_days in outlier_sensitivity_analysis:
    travel_days = n_days
    
    ils_vrp = ILS_VRP(random_state=100)
    results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_ils_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["ils_vrp"] = num_outlier
    
    ils_sa_vrp = ILS_SA_VRP(random_state=100)
    results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_ils_sa_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["ils_sa_vrp"] = num_outlier
    
    ils_ts_vrp = ILS_TS_VRP(random_state=100)
    results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_ils_ts_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["ils_ts_vrp"] = num_outlier
    
    ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
    results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_ils_sa_ts_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["ils_sa_ts_vrp"] = num_outlier
    
    mga_vrp = MGA_VRP(random_state=100)
    results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_mga_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["mga_vrp"] = num_outlier

In [10]:
outlier_sensitivity_analysis

{1: {'ils_vrp': 1,
  'ils_sa_vrp': 0,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 0,
  'mga_vrp': 0},
 2: {'ils_vrp': 0,
  'ils_sa_vrp': 0,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 0,
  'mga_vrp': 0},
 3: {'ils_vrp': 1,
  'ils_sa_vrp': 1,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 0,
  'mga_vrp': 1},
 4: {'ils_vrp': 1,
  'ils_sa_vrp': 1,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 2,
  'mga_vrp': 1},
 5: {'ils_vrp': 1,
  'ils_sa_vrp': 1,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 1,
  'mga_vrp': 0},
 6: {'ils_vrp': 1,
  'ils_sa_vrp': 2,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 3,
  'mga_vrp': 1},
 7: {'ils_vrp': 2,
  'ils_sa_vrp': 2,
  'ils_ts_vrp': 1,
  'ils_sa_ts_vrp': 1,
  'mga_vrp': 1},
 8: {'ils_vrp': 3,
  'ils_sa_vrp': 3,
  'ils_ts_vrp': 3,
  'ils_sa_ts_vrp': 3,
  'mga_vrp': 1},
 9: {'ils_vrp': 3,
  'ils_sa_vrp': 3,
  'ils_ts_vrp': 3,
  'ils_sa_ts_vrp': 3,
  'mga_vrp': 1},
 10: {'ils_vrp': 4,
  'ils_sa_vrp': 4,
  'ils_ts_vrp': 3,
  'ils_sa_ts_vrp': 3,
  'mga_vrp': 3},
 11: {'ils_vrp': 3,
  'ils_sa_vrp': 4,


In [11]:
result = pd.DataFrame()
for i in outlier_sensitivity_analysis:
    if i not in range(min_days,max_days+1):
        continue
    else:
        travel_days = i
        methods = []
        num_outlier = []
        for j in outlier_sensitivity_analysis[i]:
            methods.append(j)
            num_outlier.append(outlier_sensitivity_analysis[i][j])
        temp_res = pd.DataFrame({'methods':methods,
                                'num_outlier':num_outlier})
        temp_res['travel_days'] = travel_days
        result = pd.concat([result,temp_res])

In [12]:
result

,methods,num_outlier,travel_days
0,ils_vrp,1,1
1,ils_sa_vrp,0,1
2,ils_ts_vrp,1,1
3,ils_sa_ts_vrp,0,1
4,mga_vrp,0,1
...,...,...,...
0,ils_vrp,5,13
1,ils_sa_vrp,5,13
2,ils_ts_vrp,5,13
3,ils_sa_ts_vrp,5,13


In [13]:
result.to_excel('ils_outlier_sensitivity.xlsx',index=False)

## General Performance (50 random nodes)

In [30]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            if len(res['daily_fitness']) == 2:
                res['daily_fitness'].append(0)
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [15]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "ils_vrp": [],
    "ils_sa_vrp": [],
    "ils_ts_vrp": [],
    "ils_sa_ts_vrp": [],
    "mga_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    ils_vrp = ILS_VRP(random_state=100)
    results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_vrp"].append(results_ils_vrp)
    
    ils_sa_vrp = ILS_SA_VRP(random_state=100)
    results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_vrp"].append(results_ils_sa_vrp)
    
    ils_ts_vrp = ILS_TS_VRP(random_state=100)
    results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_ts_vrp"].append(results_ils_ts_vrp)
    
    ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
    results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)
    
    mga_vrp = MGA_VRP(random_state=100)
    results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["mga_vrp"].append(results_mga_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [16]:
random_node_analysis

{'ils_vrp': [{'solution': [[65, 48, 22, 50, 52, 93],
    [31, 66, 9, 15, 30, 2, 7, 10],
    [83, 4]],
   'fitness': 0.5640839739110178,
   'daily_fitness': [0.6272667714884695,
    0.6114539657582111,
    0.5696737421383647],
   'days_needed': 3,
   'poi_included': 16,
   'avg_rating': 4.543749999999999,
   'total_tarif': 102500,
   'total_duration': 130320,
   'run_time': 0.9756009578704834},
  {'solution': [[46, 22, 64, 34, 51, 21],
    [10, 97, 66, 48, 32, 2, 74, 57],
    [58, 89, 44, 49, 36]],
   'fitness': 0.5703359410489514,
   'daily_fitness': [0.5973963454605047,
    0.5456120943952802,
    0.5842059980334317],
   'days_needed': 3,
   'poi_included': 19,
   'avg_rating': 4.563157894736841,
   'total_tarif': 127500,
   'total_duration': 123960,
   'run_time': 2.262507915496826},
  {'solution': [[64, 93, 22, 43, 36, 15],
    [29, 48, 57, 32, 33, 74],
    [8, 46, 2, 68, 53, 5, 50, 3]],
   'fitness': 0.5592959921309482,
   'daily_fitness': [0.5410448677616619,
    0.503751835110493

In [31]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"ils_random_nodes_alldoi_1.xlsx")

### Travel Duration 1, other 0

In [32]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "ils_vrp": [],
    "ils_sa_vrp": [],
    "ils_ts_vrp": [],
    "ils_sa_ts_vrp": [],
    "mga_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    ils_vrp = ILS_VRP(random_state=100)
    results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_vrp"].append(results_ils_vrp)
    
    ils_sa_vrp = ILS_SA_VRP(random_state=100)
    results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_vrp"].append(results_ils_sa_vrp)
    
    ils_ts_vrp = ILS_TS_VRP(random_state=100)
    results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_ts_vrp"].append(results_ils_ts_vrp)
    
    ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
    results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)
    
    mga_vrp = MGA_VRP(random_state=100)
    results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["mga_vrp"].append(results_mga_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [33]:
random_node_analysis

{'ils_vrp': [{'solution': [[65, 48, 31, 66, 22],
    [50, 9, 52, 93, 15, 2, 30, 7],
    [64, 55, 72, 4, 83, 74]],
   'fitness': 0.5760416666666667,
   'daily_fitness': [0.4210648148148148,
    0.47731481481481475,
    0.437037037037037],
   'days_needed': 3,
   'poi_included': 19,
   'avg_rating': 4.563157894736841,
   'total_tarif': 207500,
   'total_duration': 124740,
   'run_time': 1.5047273635864258},
  {'solution': [[46, 64, 34, 66, 22, 10, 21, 36],
    [51, 97, 48, 2, 32, 74],
    [58, 67, 57, 5, 89, 49]],
   'fitness': 0.5816358024691358,
   'daily_fitness': [0.46875, 0.4349537037037037, 0.42800925925925926],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.545,
   'total_tarif': 192500,
   'total_duration': 130020,
   'run_time': 1.1801319122314453},
  {'solution': [[64, 29, 33, 93, 43],
    [8, 57, 48, 46, 22, 32, 2, 3],
    [74, 53, 85, 50, 5, 36, 15]],
   'fitness': 0.5919753086419752,
   'daily_fitness': [0.41365740740740736,
    0.467824074074074,
    0.47013

In [34]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"ils_random_nodes_dwaktu_1.xlsx")

### Tarif 1, other 0

In [35]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "ils_vrp": [],
    "ils_sa_vrp": [],
    "ils_ts_vrp": [],
    "ils_sa_ts_vrp": [],
    "mga_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    ils_vrp = ILS_VRP(random_state=100)
    results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_vrp"].append(results_ils_vrp)
    
    ils_sa_vrp = ILS_SA_VRP(random_state=100)
    results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_vrp"].append(results_ils_sa_vrp)
    
    ils_ts_vrp = ILS_TS_VRP(random_state=100)
    results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_ts_vrp"].append(results_ils_ts_vrp)
    
    ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
    results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)
    
    mga_vrp = MGA_VRP(random_state=100)
    results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["mga_vrp"].append(results_mga_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [36]:
random_node_analysis

{'ils_vrp': [{'solution': [[22, 48, 9, 52, 65, 93],
    [31, 66, 15, 50, 2, 30, 83],
    [90, 64, 34, 4, 7, 35, 88]],
   'fitness': 0.7017295597484277,
   'daily_fitness': [0.519811320754717,
    0.5034591194968554,
    0.5468553459119497],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.554999999999999,
   'total_tarif': 139500,
   'total_duration': 126960,
   'run_time': 1.7564074993133545},
  {'solution': [[46, 64, 34, 51, 66, 10],
    [97, 22, 21, 48, 2, 32, 74, 36],
    [58, 89, 18, 57, 49]],
   'fitness': 0.703834808259587,
   'daily_fitness': [0.5424778761061947,
    0.4828908554572271,
    0.5656342182890856],
   'days_needed': 3,
   'poi_included': 19,
   'avg_rating': 4.552631578947367,
   'total_tarif': 127500,
   'total_duration': 125700,
   'run_time': 2.9537250995635986},
  {'solution': [[57, 29, 93, 43, 2],
    [33, 64, 48, 46, 22, 32, 36],
    [8, 53, 74, 50, 3, 85, 15]],
   'fitness': 0.7187686193959375,
   'daily_fitness': [0.5381316643486324,
    0.526

In [37]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"ils_random_nodes_dtarif_1.xlsx")

### Rating 1, other 0

In [38]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "ils_vrp": [],
    "ils_sa_vrp": [],
    "ils_ts_vrp": [],
    "ils_sa_ts_vrp": [],
    "mga_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    ils_vrp = ILS_VRP(random_state=100)
    results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_vrp"].append(results_ils_vrp)
    
    ils_sa_vrp = ILS_SA_VRP(random_state=100)
    results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_vrp"].append(results_ils_sa_vrp)
    
    ils_ts_vrp = ILS_TS_VRP(random_state=100)
    results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_ts_vrp"].append(results_ils_ts_vrp)
    
    ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
    results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)
    
    mga_vrp = MGA_VRP(random_state=100)
    results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["mga_vrp"].append(results_mga_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [39]:
random_node_analysis

{'ils_vrp': [{'solution': [[93, 65, 22, 48, 66],
    [9, 31, 52, 15, 50, 2, 30, 83],
    [64, 74, 72, 7, 4, 35, 88]],
   'fitness': 0.7708333333333333,
   'daily_fitness': [0.438888888888889,
    0.4666666666666668,
    0.5531746031746029],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.574999999999999,
   'total_tarif': 212500,
   'total_duration': 126540,
   'run_time': 2.1384592056274414},
  {'solution': [[46, 64, 34, 66, 10, 51],
    [97, 21, 48, 22, 2, 32, 74, 57],
    [6, 58, 44, 5, 89, 36]],
   'fitness': 0.7288657407407404,
   'daily_fitness': [0.4166666666666664,
    0.4083333333333332,
    0.40000000000000024],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.589999999999999,
   'total_tarif': 165500,
   'total_duration': 130800,
   'run_time': 1.6405994892120361},
  {'solution': [[64, 57, 29, 43, 2],
    [8, 46, 22, 93, 33, 50, 36],
    [48, 32, 53, 5, 74, 3, 15]],
   'fitness': 0.6820506126427177,
   'daily_fitness': [0.2690476190476195,
    0.

In [40]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"ils_random_nodes_drating_1.xlsx")

## Random DOI Combinations

In [42]:
def result_to_xlsx_rd_doi(result,random_doi,random_ls,filename):
    df_doi = pd.DataFrame()
    for doi_res in range(len(result)):
        df = pd.DataFrame()
        for method in result[doi_res]:
            res_method = result[doi_res][method]
            df_method = {
                "random_seed" : random_ls,
                "fitness" : [],
                "fitness_1" : [],
                "fitness_2" : [],
                "fitness_3" : [],
                "poi_included" : [],
                "rating" : [],
                "tarif" : [],
                "total_duration" : [],
                "run_time" : [],
            }
            for res in res_method:
                if len(res['daily_fitness']) == 2:
                    res['daily_fitness'].append(0)
                df_method["fitness"].append(res['fitness'])
                df_method["fitness_1"].append(res['daily_fitness'][0])
                df_method["fitness_2"].append(res['daily_fitness'][1])
                df_method["fitness_3"].append(res['daily_fitness'][2])
                df_method["poi_included"].append(res['poi_included'])
                df_method["rating"].append(res['avg_rating'])
                df_method["tarif"].append(res['total_tarif'])
                df_method["total_duration"].append(res['total_duration'])
                df_method["run_time"].append(res['run_time'])
            df_method = pd.DataFrame(df_method)
            df_method['method'] = method
            df = pd.concat([df,df_method])
        
        doi = random_doi[doi_res]
        df['dwaktu'] = doi[0]
        df['dtarif'] = doi[1]
        df['drating'] = doi[2]
        df = df[['dwaktu','dtarif','drating','method','random_seed','fitness','fitness_1','fitness_2','fitness_3',
                'poi_included','rating','tarif','total_duration','run_time']]
        df_doi = pd.concat([df_doi,df])
#     return df_doi
    df_doi.to_excel(filename,index=False)

In [43]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [44]:
doi_combi

[(0.15, 0.45, 0.77),
 (0.71, 0.73, 0.43),
 (0.8, 0.53, 0.08),
 (0.46, 0.05, 0.93),
 (0.95, 0.34, 0.31),
 (0.77, 0.2, 0.18),
 (0.19, 0.35, 0.63),
 (0.96, 0.21, 0.96),
 (0.56, 0.9, 0.82),
 (0.16, 0.65, 0.12)]

In [45]:
doi_combi_analysis = []
for doi in doi_combi:
    print("doi combination = ",doi)
    random_ls = list(range(5,(5*50)+1,5))
    random_node_analysis ={
        "ils_vrp": [],
        "ils_sa_vrp": [],
        "ils_ts_vrp": [],
        "ils_sa_ts_vrp": [],
        "mga_vrp": []
    }
    for i in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
        travel_days = 3
        dwaktu,dtarif,drating = doi

        print("random state = ",random_ls[i])

        ils_vrp = ILS_VRP(random_state=100)
        results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ils_vrp"].append(results_ils_vrp)

        ils_sa_vrp = ILS_SA_VRP(random_state=100)
        results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ils_sa_vrp"].append(results_ils_sa_vrp)

        ils_ts_vrp = ILS_TS_VRP(random_state=100)
        results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ils_ts_vrp"].append(results_ils_ts_vrp)

        ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
        results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)

        mga_vrp = MGA_VRP(random_state=100)
        results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["mga_vrp"].append(results_mga_vrp)
    
    doi_combi_analysis.append(random_node_analysis)

doi combination =  (0.15, 0.45, 0.77)
random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
ra

random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250
doi combination =  (0.16, 0.65, 0.12)


In [46]:
doi_combi_analysis

[{'ils_vrp': [{'solution': [[65, 93, 48, 66, 52, 9, 50],
     [31, 22, 2, 15, 30, 83, 7],
     [74, 34, 51, 55, 4, 88]],
    'fitness': 0.7226045299649634,
    'daily_fitness': [0.5682358977080737,
     0.4782533458942584,
     0.5728435973191568],
    'days_needed': 3,
    'poi_included': 20,
    'avg_rating': 4.5649999999999995,
    'total_tarif': 115000,
    'total_duration': 128520,
    'run_time': 3.956449031829834},
   {'solution': [[46, 22, 51, 34, 10, 66],
     [64, 97, 48, 21, 2, 32, 89, 36],
     [6, 74, 57, 67, 44, 49, 71]],
    'fitness': 0.6937021405243832,
    'daily_fitness': [0.507113355654299,
     0.45781804264217174,
     0.46710484008148373],
    'days_needed': 3,
    'poi_included': 21,
    'avg_rating': 4.557142857142857,
    'total_tarif': 155500,
    'total_duration': 127980,
    'run_time': 1.8028249740600586},
   {'solution': [[64, 46, 57, 29, 43],
     [8, 33, 93, 48, 22, 2, 3],
     [74, 32, 20, 53, 5, 50, 36, 15, 85]],
    'fitness': 0.6783760506223239,
   

In [48]:
result_to_xlsx_rd_doi(doi_combi_analysis,doi_combi,random_ls,"ils_random_doi_combi.xlsx")

## 87 POIs

In [5]:
def result_to_xlsx_all_poi(result,n,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        for j in range(len(result[i])):
            temp = pd.DataFrame(result[i][j],columns=["fitness","run_time"])
            temp['n'] = n[j]
            temp['method'] = i
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [ ]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "ils_vrp": [],
    "ils_sa_vrp": [],
    "ils_ts_vrp": [],
    "ils_sa_ts_vrp": [],
    "mga_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(5,(5*50)+1,5))
    random_node = {
        "ils_vrp": [],
        "ils_sa_vrp": [],
        "ils_ts_vrp": [],
        "ils_sa_ts_vrp": [],
        "mga_vrp": []
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        ils_vrp = ILS_VRP(random_state=100)
        results_ils_vrp = get_analysis(ils_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ils_vrp"].append(results_ils_vrp)

        ils_sa_vrp = ILS_SA_VRP(random_state=100)
        results_ils_sa_vrp = get_analysis(ils_sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ils_sa_vrp"].append(results_ils_sa_vrp)

        ils_ts_vrp = ILS_TS_VRP(random_state=100)
        results_ils_ts_vrp = get_analysis(ils_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ils_ts_vrp"].append(results_ils_ts_vrp)

        ils_sa_ts_vrp = ILS_SA_TS_VRP(random_state=100)
        results_ils_sa_ts_vrp = get_analysis(ils_sa_ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["ils_sa_ts_vrp"].append(results_ils_sa_ts_vrp)

        mga_vrp = MGA_VRP(random_state=100)
        results_mga_vrp = get_analysis(mga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["mga_vrp"].append(results_mga_vrp)
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  10
rando

random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  50
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state

In [ ]:
run_time_analysis

In [ ]:
result_to_xlsx_all_poi(run_time_analysis,n,random_ls,"ils_various_numbers_pois_all_1.xlsx")